### Генерация имён при помощи реукррентных нейронных сетей.

Мы попробуем разобраться в устройстве рекуррентных нейронных сетей, рассмотрев ряд "игрушечных" примеров задач.

У вас возникали проблемы с поиском переменной при программировании? Посмотрим, насколько сложно придумать имя для сына или дочери. Очевидно, люди недостаточно хорошо разбираются в том, какое имя подойдёт для ребёнка, поэтому мы доверим эту задачу рекуррентной нейронной сети.

Начнём с импорта необходимых библиотек.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Данные
Набор данных содержит примерно 8 тысяч имён землян из разных культур, написанных латиницей.

In [ ]:
import os
start_token = " "

with open("names.txt") as f:
    lines = f.read()[:-1].split('\n')
    lines = [start_token + line for line in lines]

In [ ]:
print ('n samples = ',len(lines))
for x in lines[::1000]:
    print (x)
    


In [ ]:
MAX_LENGTH = max(map(len, lines))
print("max length =", MAX_LENGTH)

plt.title('Sequence length distribution')
plt.hist(list(map(len, lines)),bins=25);

# Обработка текста

Сначала нам нужно собрать "словарь" уникальных знаков, или уникальных символов. После этого входные данные могут быть представленны как последовательности индексов символов.

In [ ]:
#all unique characters go here
tokens = <all unique characters in the dataset>

tokens = list(tokens)

num_tokens = len(tokens)
print ('num_tokens = ', num_tokens)

assert 50 < num_tokens < 60, "Names should contain within 50 and 60 unique tokens depending on encoding"

### Перевод символов в целые числа

Torch создан для обработки чисел, а не строк.
Для обучения нейронной сети необходимо заменить символы на их индексы в общем списке символов.

Создадим словарь, который позволит осуществить подобное преобразование.

In [ ]:
token_to_id = <dictionary of symbol -> its identifier (index in tokens list)>

In [ ]:
assert len(tokens) == len(token_to_id), "dictionaries must have same size"

for i in range(num_tokens):
    assert token_to_id[tokens[i]] == i, "token identifier must be it's position in tokens list"

print("Seems alright!")

In [ ]:
def to_matrix(lines, max_len=None, pad=token_to_id[' '], dtype='int32', batch_first = True):
    """Casts a list of names into rnn-digestable matrix"""
    
    max_len = max_len or max(map(len, lines))
    lines_ix = np.zeros([len(lines), max_len], dtype) + pad

    for i in range(len(lines)):
        line_ix = [token_to_id[c] for c in lines[i]]
        lines_ix[i, :len(line_ix)] = line_ix
        
    if not batch_first: # convert [batch, time] into [time, batch]
        lines_ix = np.transpose(lines_ix)

    return lines_ix

In [ ]:
#Премер: преобразование 4 случайных имён в матрицы, дозаполнение нулями
print('\n'.join(lines[::2000]))
print(to_matrix(lines[::2000]))

# Рекуррентная нейронная сеть

Рекуррентная нейронная сеть может быть представлена как последовательное применение полносвязного слоя к входным данным $x_t$ и предыдущему состоянию РНН $h_t$. Именно это мы и сделаем.
<img src="./rnn.png" width=480>

Поскольку мы обучаем языковую модель, в ней должны быть:
* Слой представления (embedding), который преобразует идентификатор символа x_t в вектор.
* Выходной слой, предсказывающий вероятности следующей фонемы.

In [ ]:
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class CharRNNCell(nn.Module):
    """
    Implement the scheme above as torch module
    """
    def __init__(self, num_tokens=len(tokens), embedding_size=16, rnn_num_units=64):
        super(self.__class__,self).__init__()
        self.num_units = rnn_num_units
        
        self.embedding = nn.Embedding(num_tokens, embedding_size)
        self.rnn_update = nn.Linear(embedding_size + rnn_num_units, rnn_num_units)
        self.rnn_to_logits = nn.Linear(rnn_num_units, num_tokens)
        
    def forward(self, x, h_prev):
        """
        This method computes h_next(x, h_prev) and log P(x_next | h_next)
        We'll call it repeatedly to produce the whole sequence.
        
        :param x: batch of character ids, variable containing vector of int64
        :param h_prev: previous rnn hidden states, variable containing matrix [batch, rnn_num_units] of float32
        """
        # get vector embedding of x
        x_emb = self.embedding(x)
        
        # compute next hidden state using self.rnn_update
        # hint: use torch.cat(..., dim=...) for concatenation
        h_next = ###YOUR CODE HERE
        
        h_next = F.tanh(h_next)
        
        assert h_next.size() == h_prev.size()
        
        #compute logits for next character probs
        logits = ###YOUR CODE
        
        return h_next, F.log_softmax(logits, -1)
    
    def initial_state(self, batch_size):
        """ return rnn state before it processes first input (aka h0) """
        return Variable(torch.zeros(batch_size, self.num_units))

In [ ]:
char_rnn = CharRNNCell()

### Цикл РНН

После того как мы определили единичный шаг РНН, мы можем циклично применять его для получения предсказаний для каждого шага.

In [ ]:
def rnn_loop(char_rnn, batch_ix):
    """
    Computes log P(next_character) for all time-steps in lines_ix
    :param lines_ix: an int32 matrix of shape [batch, time], output of to_matrix(lines)
    """
    batch_size, max_length = batch_ix.size()
    hid_state = char_rnn.initial_state(batch_size)
    logprobs = []

    for x_t in batch_ix.transpose(0,1):
        hid_state, logp_next = char_rnn(x_t, hid_state)  # <-- here we call your one-step code
        logprobs.append(logp_next)
        
    return torch.stack(logprobs, dim=1)

In [ ]:
batch_ix = to_matrix(lines[:5])
batch_ix = Variable(torch.LongTensor(batch_ix))

logp_seq = rnn_loop(char_rnn, batch_ix)

assert isinstance(logp_seq, Variable) and torch.max(logp_seq).data.numpy() <= 0
assert tuple(logp_seq.size()) ==  batch_ix.size() + (num_tokens,)

### Функция правдоподобия и градиенты

Мы можем обучить нейронную сеть минимизировать перекрёстную энтропию (максимизировать логарифмическую функцию правдоподобия) с истинным распределением следующих символов.

Чтобы осуществить подобный расчёт в векторизированной форме, возьмём `batch_ix[:, 1:]` - мартицу идентификаторов символов, смещённых на i шагов влево. Таким образом, i-й символ будет являться следующим символом для i-ой операции предсказания.

In [ ]:
predictions_logp = logp_seq[:, :-1]
actual_next_tokens = batch_ix[:, 1:]

logp_next = torch.gather(predictions_logp, dim=2, index=actual_next_tokens[:,:,None])

loss = -logp_next.mean()

In [ ]:
loss.backward()

In [ ]:
for w in char_rnn.parameters():
    assert w.grad is not None and torch.max(torch.abs(w.grad)).data.numpy() != 0, \
        "Loss is not differentiable w.r.t. a weight with shape %s. Check forward method." % (w.size(),)

### Цикл обучения

Мы будем обучать символьную РНН точно так же, как и любую другую модель глубинного обучения: стохастическим градиентным спуском для минибатчей.

In [ ]:
from IPython.display import clear_output
from random import sample

char_rnn = CharRNNCell()
opt = torch.optim.Adam(char_rnn.parameters())
history = []

In [ ]:

for i in range(1000):
    batch_ix = to_matrix(sample(lines, 32), max_len=MAX_LENGTH)
    batch_ix = Variable(torch.LongTensor(batch_ix))
    
    logp_seq = rnn_loop(char_rnn, batch_ix)
    
    # compute loss
    <YOUR CODE>
    
    loss = ###YOUR CODE
    
    # train with backprop
    <YOUR CODE>
    
    history.append(loss.data.numpy()[0])
    if (i+1)%100==0:
        clear_output(True)
        plt.plot(history,label='loss')
        plt.legend()
        plt.show()

assert np.mean(history[:10]) > np.mean(history[-10:]), "RNN didn't converge."

### РНН: сэмплирование
После обучения РНН можно приступать к генерации примеров.
Для этого пригодится функция шага РНН, определённая в `char_rnn.forward`.

In [ ]:
def generate_sample(char_rnn, seed_phrase=' ', max_length=MAX_LENGTH, temperature=1.0):
    '''
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs,
                        smaller temperature converges to the single most likely output
    '''
    
    x_sequence = [token_to_id[token] for token in seed_phrase]
    x_sequence = Variable(torch.LongTensor([x_sequence]))
    hid_state = char_rnn.initial_state(batch_size=1)
    
    #feed the seed phrase, if any
    for i in range(len(seed_phrase) - 1):
        hid_state, _ = char_rnn(x_sequence[:, i], hid_state)
    
    #start generating
    for _ in range(max_length - len(seed_phrase)):
        hid_state, logp_next = char_rnn(x_sequence[:, -1], hid_state)
        p_next = F.softmax(logp_next / temperature, -1).data.numpy()[0]
        
        # sample next token and push it back into x_sequence
        next_ix = np.random.choice(num_tokens,p=p_next)
        next_ix = Variable(torch.LongTensor([[next_ix]]))
        x_sequence = torch.cat([x_sequence, next_ix], dim=1)
        
    return ''.join([tokens[ix] for ix in x_sequence.data.numpy()[0]])

In [ ]:
for _ in range(10):
    print(generate_sample(char_rnn))

In [ ]:
for _ in range(50):
    print(generate_sample(char_rnn, seed_phrase=' Deep'))

### Попробуйте!
Вы только что создали рекуррентную языковую модель, способную осуществлять генерацию любого вида последовательностей. Есть много видов данных, к которым она может быть применена:

* Рассказы/стихи/песни вашего любимого автора
* Новостные заголовки
* Исходный код Linux или Tensorflow
* Молекулы в формате [smiles](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)
* Музыка в виде нот/аккордов
* Названия из каталогов IKEA
* Имена покемонов
* Карты из Magic, the Gathering / Hearthstone

Если вы хотите попробовать что-то из этого, обратите внимание на:
* Сейчас данные представлены в виде последовательности строк, а рассказ может быть представлен как последовательность предложений. Также можно изменить весь процесс предварительной обработки данных.
* Некоторые датасеты находятся в свободном доступе, другие могут быть собраны из открытых источников. Попробуйте использовать `Selenium` или `Scrapy` для этого.
* Проверьте, что MAX_LENGTH корректно настроена для датасетов большего размера. Ниже представлен дополнительный раздел о динамических РНН.
* Более сложные задачи требуют большей архитектуры РНН. Попробуйте использовать больше нейронов или больше слоёв. Также может потребоваться больше итераций обучения.
* Долгосрочные зависимости в музыке, рассказах или молекулярной структуре лучше обрабатывать при помощи LSTM или GRU

### Более серьёзно

Только что мы вручную создали низкоуровневую имплементацию РНН. Тем не менее, вряд ли стоит переписывать её с нуля для каждой задачи.

Как вы могли догадаться, в Torch есть решение данной проблемы. Присутствуют две опции:
* `nn.RNNCell(emb_size, rnn_num_units)` - представляет собой единичный шаг РНН наподобие имплементированного выше. По сути, является комбинацией слоёв concat, linear и tanh.
* `nn.RNN(emb_size, rnn_num_units)` - Полностью реализует цикл РНН.

Другими опциями являются `nn.LSTMCell` или `nn.LSTM`, `nn.GRUCell` или `nn.GRU` и так далее.

В данном примере мы перепишем символьную РНН и цикл РНН используя API высокого уровня.

In [ ]:
class CharRNNLoop(nn.Module):
    def __init__(self, num_tokens=num_tokens, emb_size=16, rnn_num_units=64):
        super(self.__class__, self).__init__()
        self.emb = nn.Embedding(num_tokens, emb_size)
        self.rnn = nn.RNN(emb_size, rnn_num_units, batch_first=True)
        self.hid_to_logits = nn.Linear(rnn_num_units, num_tokens)
        
    def forward(self, x):
        assert isinstance(x, Variable) and isinstance(x.data, torch.LongTensor)
        h_seq, _ = self.rnn(self.emb(x))
        next_logits = self.hid_to_logits(h_seq)
        next_logp = F.log_softmax(next_logits, dim=-1)
        return next_logp
    
model = CharRNNLoop()

In [ ]:
# the model applies over the whole sequence
batch_ix = to_matrix(sample(lines, 32), max_len=MAX_LENGTH)
batch_ix = Variable(torch.LongTensor(batch_ix))

logp_seq = model(batch_ix)

# compute loss. This time we use nll_loss with some duct tape
loss = F.nll_loss(logp_seq[:, 1:].contiguous().view(-1, num_tokens), 
                  batch_ix[:, :-1].contiguous().view(-1))

loss.backward()

Другой пример:

In [ ]:
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class CharLSTMCell(nn.Module):
    """
    Implements something like CharRNNCell, but with LSTM
    """
    def __init__(self, num_tokens=len(tokens), embedding_size=16, rnn_num_units=64):
        super(self.__class__,self).__init__()
        self.num_units = rnn_num_units
        self.emb = nn.Embedding(num_tokens, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, rnn_num_units)
        self.rnn_to_logits = nn.Linear(rnn_num_units, num_tokens)
        
    def forward(self, x, prev_state):
        (prev_h, prev_c) = prev_state
        (next_h, next_c) = self.lstm(self.emb(x), (prev_h, prev_c))
        logits = self.rnn_to_logits(next_h)
        
        return (next_h, next_c), F.log_softmax(logits, -1)
    
    def initial_state(self, batch_size):
        """ LSTM has two state variables, cell and hid """
        return Variable(torch.zeros(batch_size, self.num_units)), Variable(torch.zeros(batch_size, self.num_units))
    
char_lstm = CharLSTMCell()

In [ ]:
opt = torch.optim.Adam(char_rnn.parameters())
history = []
cross_entropy = nn.CrossEntropyLoss()

for i in range(1000):
    batch_ix = to_matrix(sample(lines, 32), max_len=MAX_LENGTH)
    batch_ix = Variable(torch.LongTensor(batch_ix.tolist()))

    logp_seq = rnn_loop(char_lstm, batch_ix)

    # compute loss. This time we use nll_loss with some duct tape
    loss = F.nll_loss(logp_seq[:, 1:].contiguous().view(-1, num_tokens), 
                      batch_ix[:, :-1].contiguous().view(-1))

    # train with backprop
    char_rnn.zero_grad()
    loss.backward()
    opt.step()
    
#     print(loss.size())
#     assert loss.size()
    history.append(loss.data.numpy())
    if (i+1)%100==0:
        clear_output(True)
        plt.plot(history,label='loss')
        plt.legend()
        plt.show()

assert np.mean(history[:10]) > np.mean(history[-10:]), "RNN didn't converge."

In [ ]:
for _ in range(10):
    print(generate_sample(char_rnn))

__Дополнительное задание: __ Имплементируйте модель, использующую два слоя LSTM (вторая LSTM использует результат первой в качестве входных значений) и обучите её на ваших данных.